In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Carica i dati
data = pd.read_csv('estrz.csv')
print(data.columns)
data.columns = data.columns.str.strip()

# Prepara i dati
data = data[['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6']]

# Normalizzazione dei dati
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Creazione delle sequenze per l'addestramento
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Imposta la lunghezza della sequenza (ad esempio, 10 giorni alterni)
seq_length = 10
X, y = create_sequences(data_scaled, seq_length)

# Dividi il dataset in addestramento e test
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Crea il modello LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(6))
model.compile(optimizer='adam', loss='mse')

# Addestra il modello
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=5000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Previsioni future
def predict_future(model, last_sequence, n_predictions):
    current_seq = last_sequence
    predictions = []
    for _ in range(n_predictions):
        prediction = model.predict(current_seq.reshape(1, seq_length, 6))
        predictions.append(prediction[0])
        current_seq = np.vstack((current_seq[1:], prediction))
    return scaler.inverse_transform(predictions)

# Prevedi i prossimi 5 giorni alterni
last_sequence = X[-1]  # Prendi l'ultima sequenza
n_predictions = 5  # Numero di record futuri da prevedere
future_predictions = predict_future(model, last_sequence, n_predictions)

# Converti le previsioni in interi arrotondati
future_predictions_rounded = np.round(future_predictions).astype(int)

# Visualizza le previsioni
future_df = pd.DataFrame(future_predictions_rounded, columns=['Numero1', 'Numero2', 'Numero3', 'Numero4', 'Numero5', 'Numero6'])
print(future_df)
